# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
from sqlalchemy import create_engine
import pandas as pd
import re
from datetime import datetime
from nltk.corpus import stopwords
from nltk import WordNetLemmatizer, RegexpTokenizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, \
    GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.metrics import accuracy_score, recall_score
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.base import BaseEstimator, TransformerMixin

In [2]:
# load data from database
def load_data(db_name: str):
    """
    Function used to load data from sqlite database
    :param db_name: database file name
    :return: X and Y
    """
    engine = create_engine(f'sqlite:///{db_name}.db')
    table_name = engine.table_names()[0]
    df = pd.read_sql(f'select * from {table_name}', engine, index_col='id')
    not_in_Y = ['original', 'genre', 'message']
    X = df['message'].values
    Y = df[[col for col in df.columns if col not in not_in_Y]].values


    return X, Y


X, Y = load_data('DisasterResponse')

### 2. Write a tokenization function to process your text data

In [37]:
def clean_text(text: str):
    """
    Function used to clean text data:
    - text normalization
    - text tokenization
    - text lemmatization or stemming
    :param text: text to clean
    :param reduce_word: lammatizer or stemming object
    :return: clean text
    """
    # text normalization and tokenization
    tokenizer = RegexpTokenizer(r'[\w]')
    tokens = tokenizer.tokenize(text)

    # removing stopwords
    stop_words = stopwords.words('english')
    tokens = [word.strip() for word in tokens if word not in stop_words]

    # lemmatization or stemming
    lemmatizer = WordNetLemmatizer()
    text = [lemmatizer.lemmatize(word) for word in tokens]

    return text

### 3. Train pipeline
- Split data into train and test sets
- Train pipeline

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3,
                                                    random_state=0)

### 4. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [39]:
pipeline_rf = Pipeline([
    ('count_vectorizer', CountVectorizer(tokenizer=clean_text)),
    ('tfids', TfidfTransformer()),
    ('cls',
     MultiOutputClassifier(estimator=RandomForestClassifier()))
])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [40]:
start = datetime.now()

pipeline_rf.fit(X_train, y_train)
y_predict = pipeline_rf.predict(X_test)

end = datetime.now()
delta = (end - start).seconds
print(f'It took {delta // 60} minutes and {delta % 60} seconds')

It took 5 minutes and 50 seconds


In [41]:
accuracy = []

for c_r, c_p in zip(y_test.T, y_predict.T):
    accuracy.append(accuracy_score(c_r, c_p))

mean_accuracy = sum(accuracy) / len(accuracy)

print(f'The accuracy of the model is: {mean_accuracy:.2%}')

The accuracy of the model is: 93.07%


### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = {
    'tfids__norm': ['l1', 'l2'],
    'cls__estimator__n_estimators': [10, 50, 100, 500],
    'cls__estimator__max_depth': [None, 1, 2, 3, 4, 5],
    'cls__estimator__min_samples_leaf': [1, 5, 10]

}

cv = GridSearchCV(pipeline_rf, param_grid=parameters)

In [ ]:
start = datetime.now()

cv.fit(X_train, y_train)

end = datetime.now()
delta = (end - start).seconds
print(f'It took {delta // 60} minutes and {delta % 60} seconds')

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [ ]:
temp_pipeline = Pipeline([
    ('count_vectorizer',
     CountVectorizer(tokenizer=CleanDate(reduce_word=PorterStemmer))),
    ('tfids', TfidfTransformer()),
    ('cls',
     MultiOutputClassifier(estimator=RandomForestClassifier()))
])

for i in pipeline_rf.get_params().keys():
    print(i)